In [234]:
import dask
import dask.dataframe as df
import numpy as np
import pandas as pd
from scipy import sparse as sp
from math import log
from collections import Counter
adultos = df.read_csv('../data/adult_ssn.csv')
np.random.seed(0)

In [126]:
true_classes, true_idx =np.unique([1,2,3,3,2,1], return_inverse=True)
print(true_classes)
print(true_idx)
pred_classes, pred_idx =np.unique([4,5,4,4,5,5], return_inverse=True)
print(pred_classes)
print(pred_idx)
n_classes = true_classes.shape[0]
n_preds = pred_classes.shape[0]
true_classes.tolist()

[1 2 3]
[0 1 2 2 1 0]
[4 5]
[0 1 0 0 1 1]


[1, 2, 3]

In [127]:
contingency = sp.coo_matrix((np.ones(true_idx.shape[0]),
                                 (true_idx, pred_idx)),
                                shape=(n_classes, n_preds),
                                dtype=np.int)
print(contingency)

  (0, 0)	1
  (1, 1)	1
  (2, 0)	1
  (2, 0)	1
  (1, 1)	1
  (0, 1)	1


In [253]:
nzx, nzy, nz_val = sp.find(contingency)
print(nzx)
print(nzy)
print(nz_val)
nzx

[0 2 0 1]
[0 0 1 1]
[1 2 1 2]


array([0, 2, 0, 1], dtype=int32)

In [129]:
contingency_sum = contingency.sum()
print(contingency_sum)
pi = np.ravel(contingency.sum(axis=1))
print(pi)
pj = np.ravel(contingency.sum(axis=0))
print(pj)

6
[2 2 2]
[3 3]


In [130]:
log_contingency_nm = np.log(nz_val)
print(log_contingency_nm)
contingency_nm = nz_val / contingency_sum
print(contingency_nm)
# Don't need to calculate the full outer product, just for non-zeroes
outer = pi.take(nzx).astype(np.int64) * pj.take(nzy).astype(np.int64)
print(pi.take(nzx).astype(np.int64))
print(outer)
print(pj.take(nzy).astype(np.int64))
log_outer = -np.log(outer) + log(pi.sum()) + log(pj.sum())
mi = (contingency_nm * (log_contingency_nm - log(contingency_sum)) +
          contingency_nm * log_outer)
print(mi.sum())

[0.         0.69314718 0.         0.69314718]
[0.16666667 0.33333333 0.16666667 0.33333333]
[2 2 2 2]
[6 6 6 6]
[3 3 3 3]
0.4620981203732969


In [166]:
true_classes_1, true_idx_1 =np.unique([1,2,3], return_inverse=True)
print(true_classes_1)
print(true_idx_1)
pred_classes_1, pred_idx_1 =np.unique([4,5,4], return_inverse=True)
print(pred_classes_1)
print(pred_idx_1)
n_classes_1 = true_classes_1.shape[0]
n_preds_1 = pred_classes_1.shape[0]
print()
true_classes_2, true_idx_2 =np.unique([3,2,1], return_inverse=True)
print(true_classes_2)
print(true_idx_2)
pred_classes_2, pred_idx_2 =np.unique([4,5,5], return_inverse=True)
print(pred_classes_2)
print(pred_idx_2)
n_classes_2 = true_classes_2.shape[0]
n_preds_2 = pred_classes_2.shape[0]

[1 2 3]
[0 1 2]
[4 5]
[0 1 0]

[1 2 3]
[2 1 0]
[4 5]
[0 1 1]


In [142]:
contingency_1 = sp.coo_matrix((np.ones(true_idx_1.shape[0]),
                                 (true_idx_1, pred_idx_1)),
                                shape=(n_classes_1, n_preds_1),
                                dtype=np.int)
print(contingency_1.todense())

contingency_2 = sp.coo_matrix((np.ones(true_idx_2.shape[0]),
                                 (true_idx_2, pred_idx_2)),
                                shape=(n_classes_2, n_preds_2),
                                dtype=np.int)
print(contingency_2.todense())

[[1 0]
 [0 1]
 [1 0]]
[[0 1]
 [0 1]
 [1 0]]


In [153]:
nzx_1, nzy_1, nz_val_1 = sp.find(contingency_1)
print(nzx_1)
print(nzy_1)
print(nz_val_1)
print()
nzx_2, nzy_2, nz_val_2 = sp.find(contingency_2)
print(nzx_2)
print(nzy_2)
print(nz_val_2)

[0 2 1]
[0 0 1]
[1 1 1]

[2 0 1]
[0 1 1]
[1 1 1]


In [134]:
contingency_sum = contingency_1.sum() + contingency_2.sum()
print(contingency_sum)

6


In [135]:
pi_1 = np.ravel(contingency_1.sum(axis=1))
print(pi_1)
pj_1 = np.ravel(contingency_1.sum(axis=0))
print(pj_1)

pi_2 = np.ravel(contingency_2.sum(axis=1))
print(pi_2)
pj_2 = np.ravel(contingency_2.sum(axis=0))
print(pj_2)

[1 1 1]
[2 1]
[1 1 1]
[1 2]


In [136]:
lista = [1,2,3]
lista[2]

3

In [324]:
pi_dask = [0,0,0]
for index, clase in enumerate(true_classes):
    try:
        index_clase_1 =true_classes_1.tolist().index(clase)
    except (IndexError, ValueError):
        index_clase_1 = None
    
    try:
        index_clase_2 =true_classes_2.tolist().index(clase)
    except (IndexError, ValueError):
        index_clase_2 = None
    
    if index_clase_1 is not None:
        pi_dask[index] = pi_dask[index] + pi_1[true_classes_1.tolist().index(clase)]
    if index_clase_2 is not None:
        pi_dask[index] = pi_dask[index] + pi_2[true_classes_2.tolist().index(clase)]

In [325]:
pi_dask

[2, 2, 2]

In [318]:
@dask.delayed
def gen_pi_dask(mi_delayed_list: list, true_classes_len: int):
    pi_dask = [0 for i in range(true_classes_len)]
    for index, clase in enumerate(true_classes):
        for mi_delayed in mi_delayed_list:
            try:
                index_clase =mi_delayed['true_classes'].tolist().index(clase)
            except (IndexError, ValueError):
                index_clase = None
            if index_clase is not None:
                pi_dask[index] = pi_dask[index] + mi_delayed['pi'][mi_delayed['true_classes'].tolist().index(clase)]
    return pi_dask

In [138]:
pi_dask

[2, 2, 2]

In [139]:
pj_dask = [0,0]
for index, clase in enumerate(pred_classes):
    try:
        index_clase_1 =pred_classes_1.tolist().index(clase)
    except (IndexError, ValueError):
        index_clase_1 = None
    
    try:
        index_clase_2 =pred_classes_2.tolist().index(clase)
    except (IndexError, ValueError):
        index_clase_2 = None
    
    if index_clase_1 is not None:
        pj_dask[index] = pj_dask[index] + pj_1[pred_classes_1.tolist().index(clase)]
    if index_clase_2 is not None:
        pj_dask[index] = pj_dask[index] + pj_2[pred_classes_2.tolist().index(clase)]

In [ ]:
@dask.delayed
def gen_pj_dask(mi_delayed_list: list, pred_classes_len: int):
    pj_dask = [0 for i in range(pred_classes_len)]
    for index, clase in enumerate(pred_classes):
        for mi_delayed in mi_delayed_list:
            try:
                index_clase =mi_delayed['pred_classes'].tolist().index(clase)
            except (IndexError, ValueError):
                index_clase = None
            if index_clase is not None:
                pi_dask[index] = pi_dask[index] + mi_delayed['pj'][mi_delayed['pred_classes'].tolist().index(clase)]
    return pi_dask

In [140]:
pj_dask

[3, 3]

In [192]:
true_nzx_1 = list(map(lambda x: true_classes_1[x], nzx_1.tolist()))
true_nzy_1 = list(map(lambda x: pred_classes_1[x], nzy_1.tolist()))
true_nz_val_1 = nz_val_1.tolist()
true_nzx_2 = list(map(lambda x: true_classes_1[x], nzx_2.tolist()))
true_nzy_2 = list(map(lambda x: pred_classes_2[x], nzy_2.tolist()))
true_nz_val_2 = nz_val_2.tolist()

In [181]:
cross_clusters_1 = list(zip(zip(true_nzx_1,true_nzy_1),true_nz_val_1))
cross_clusters_2 = list(zip(zip(true_nzx_2,true_nzy_2),true_nz_val_2))

In [211]:
cross_clusters = dict(Counter(dict(cross_clusters_1)) + Counter(dict(cross_clusters_2)))
cross_clusters

{(1, 4): 1, (1, 5): 1, (2, 5): 2, (3, 4): 2}

In [206]:
true_nzx_1_np = np.array(list(map(lambda x: true_classes_1[x], nzx_1)))
true_nzy_1_np = np.array(list(map(lambda x: pred_classes_1[x], nzy_1)))
true_nz_val_1 = nz_val_1
true_nzx_2_np = np.array(list(map(lambda x: true_classes_1[x], nzx_2)))
true_nzy_2_np = np.array(list(map(lambda x: pred_classes_2[x], nzy_2)))
true_nz_val_2 = nz_val_2

In [210]:
cross_clusters_1_np = list(zip(zip(true_nzx_1_np,true_nzy_1_np),true_nz_val_1))
cross_clusters_2_np = list(zip(zip(true_nzx_2_np,true_nzy_2_np),true_nz_val_2))

In [212]:
cross_clusters = dict(Counter(dict(cross_clusters_1_np)) + Counter(dict(cross_clusters_2_np)))
cross_clusters

{(1, 4): 1, (1, 5): 1, (2, 5): 2, (3, 4): 2}

In [249]:
def map_to_global_indexes(nz_val: tuple, trues, preds):
    x = np.where(trues==nz_val[0])[0].tolist()[0]
    y = np.where(preds==nz_val[1])[0].tolist()[0]
    return (x,y)

In [255]:
coords_cross = list(map(lambda x: map_to_global_indexes(x, true_classes, pred_classes),list(cross_clusters.keys())))
coords_cross

[(0, 0), (2, 0), (1, 1), (0, 1)]

In [279]:
nzx_dask, nzy_dask, nz_val_dask = np.array([], dtype=np.int64),np.array([], dtype=np.int64),np.array([], dtype=np.int64)
#for key in cross_clusters.keys():
    

In [321]:
@dask.delayed
def gen_nzx_nzy_nzval_dask(mi_delayed_list: list):
    nzx_dask, nzy_dask, nz_val_dask = np.array([], dtype=np.int64),np.array([], dtype=np.int64),np.array([], dtype=np.int64)
    cross_clusters_list = []
    for mi_delayed in mi_delayed_list:
        true_nzx_np = np.array(list(map(lambda x: mi_delayed['true_classes'][x], mi_delayed['nzx'])))
        true_nzy_np = np.array(list(map(lambda x: mi_delayed['pred_classes'][x], mi_delayed['nzy'])))
        true_nz_val = mi_delayed['nz_val']
        cross_clusters_list.append(Counter(dict(list(zip(zip(true_nzx_np,true_nzy_np),true_nz_val)))))
    cross_clusters = dict(sum(cross_clusters_list))
    #coords_cross = list(map(lambda x: map_to_global_indexes(x, true_classes, pred_classes),list(cross_clusters.keys())))
    for key in cross_clusters.keys():
        nzx_dask = np.append(nzx_dask, true_classes.tolist().index(key[0]))
        nzy_dask = np.append(nzy_dask, pred_classes.tolist().index(key[1]))
        nz_val_dask = np.append(nz_val_dask, cross_clusters[key])
    return nzx_dask, nzy_dask, nz_val_dask

In [281]:
for key in cross_clusters.keys():
    nzx_dask = np.append(nzx_dask, true_classes.tolist().index(key[0]))
    nzy_dask = np.append(nzy_dask, pred_classes.tolist().index(key[1]))
    nz_val_dask = np.append(nz_val_dask, cross_clusters[key])

In [282]:
nzx_dask


array([0, 2, 1, 0])

In [283]:
nzy_dask

array([0, 0, 1, 1])

In [284]:
nz_val_dask

array([1, 2, 2, 1])

In [286]:
log_contingency_nm_dask = np.log(nz_val_dask)
print(log_contingency_nm_dask)
contingency_nm_dask = nz_val_dask / contingency_sum
print(contingency_nm_dask)
# Don't need to calculate the full outer product, just for non-zeroes
outer_dask = pi.take(nzx_dask).astype(np.int64) * pj.take(nzy_dask).astype(np.int64)
print(pi.take(nzx_dask).astype(np.int64))
print(outer_dask)
print(pj.take(nzy_dask).astype(np.int64))
log_outer_dask = -np.log(outer_dask) + log(sum(pi_dask)) + log(sum(pj_dask))
mi_dask = (contingency_nm_dask * (log_contingency_nm_dask - log(contingency_sum)) +
          contingency_nm_dask * log_outer_dask)
print(mi_dask.sum())

[0.         0.69314718 0.69314718 0.        ]
[0.16666667 0.33333333 0.33333333 0.16666667]
[2 2 2 2]
[6 6 6 6]
[3 3 3 3]
0.4620981203732969


In [ ]:
[0.         0.69314718 0.         0.69314718]
[0.16666667 0.33333333 0.16666667 0.33333333]
[2 2 2 2]
[6 6 6 6]
[3 3 3 3]
0.4620981203732969

In [308]:
@dask.delayed
def partition_mutual_info_score(true: pd.Series, pred: pd.Series):
    datos = {}
    true_classes, true_idx = np.unique(true, return_inverse=True)
    datos['true_classes'] = true_classes
    datos['true_idx'] = true_idx
    print(true_classes)
    print(true_idx)
    pred_classes, pred_idx = np.unique(pred, return_inverse=True)
    datos['pred_classes'] = pred_classes
    datos['pred_idx'] = pred_idx
    print(pred_classes)
    print(pred_idx)
    n_classes = true_classes.shape[0]
    n_preds = pred_classes.shape[0]
    datos['n_classes'] = n_classes
    datos['n_preds'] = n_preds
    contingency = sp.coo_matrix((np.ones(true_idx.shape[0]),
                                 (true_idx, pred_idx)),
                                shape=(n_classes, n_preds),
                                dtype=np.int)
    nzx, nzy, nz_val = sp.find(contingency)
    print(nzx)
    print(nzy)
    print(nz_val)
    datos['nzx'], datos['nzy'], datos['nz_val'] = nzx, nzy, nz_val
    contingency_sum = contingency.sum()
    print(contingency_sum)
    datos['contingency_sum'] = contingency_sum
    pi = np.ravel(contingency.sum(axis=1))
    print(pi)
    datos['pi'] = pi
    pj = np.ravel(contingency.sum(axis=0))
    print(pj)
    datos['pj'] = pj
    return datos

In [291]:
datos_1 = partition_mutual_info_score([1,2,3], [4,5,4])

[1 2 3]
[0 1 2]
[4 5]
[0 1 0]
[0 2 1]
[0 0 1]
[1 1 1]
3
[1 1 1]
[2 1]


In [292]:
datos_2 = partition_mutual_info_score([3,2,1], [4,5,5])

[1 2 3]
[2 1 0]
[4 5]
[0 1 1]
[2 0 1]
[0 1 1]
[1 1 1]
3
[1 1 1]
[1 2]


In [293]:
delayes = [datos_1,datos_2]

In [319]:
gen_pi_dask(delayes, 3).compute()

[2, 2, 2]

In [294]:
delayes

[{'contingency_sum': 3,
  'n_classes': 3,
  'n_preds': 2,
  'nz_val': array([1, 1, 1]),
  'nzx': array([0, 2, 1], dtype=int32),
  'nzy': array([0, 0, 1], dtype=int32),
  'pi': array([1, 1, 1], dtype=int64),
  'pj': array([2, 1], dtype=int64),
  'pred_classes': array([4, 5]),
  'pred_idx': array([0, 1, 0]),
  'true_classes': array([1, 2, 3]),
  'true_idx': array([0, 1, 2])},
 {'contingency_sum': 3,
  'n_classes': 3,
  'n_preds': 2,
  'nz_val': array([1, 1, 1]),
  'nzx': array([2, 0, 1], dtype=int32),
  'nzy': array([0, 1, 1], dtype=int32),
  'pi': array([1, 1, 1], dtype=int64),
  'pj': array([1, 2], dtype=int64),
  'pred_classes': array([4, 5]),
  'pred_idx': array([0, 1, 1]),
  'true_classes': array([1, 2, 3]),
  'true_idx': array([2, 1, 0])}]

In [299]:
true_serie = df.from_pandas(pd.Series([1,2,3,3,2,1]), npartitions=2)
pred_serie = df.from_pandas(pd.Series([4,4,5,5,4,5]), npartitions=2)

In [309]:
mi_delayed = [partition_mutual_info_score(true_serie.get_partition(i), pred_serie.get_partition(i)) for i in range(len(true_delayed))]

In [310]:
mi_delayed

[Delayed('partition_mutual_info_score-057f2df2-b99f-4eb8-92d1-dfbf2e226ae8'),
 Delayed('partition_mutual_info_score-a1517822-df07-4517-a6a3-c4820fdf8f52')]

In [323]:
gen_nzx_nzy_nzval_dask(mi_delayed).compute()

[1 2 3]
[1 2 3][2 1 0]

[0 1 2]
[4 5]
[4 5][0 0 1]

[1 0 1][0 1 2]

[1 0 2][0 0 1]

[0 1 1][1 1 1]

[1 1 1]3

3[1 1 1]

[1 1 1][2 1]

[1 2]


TypeError: unsupported operand type(s) for +: 'int' and 'Counter'

In [326]:
a, b,c = [1,2,3]

In [327]:
a

1